# Working with Tools in Strands Agents

Tools are what make agents truly powerful. They allow agents to interact with external systems, perform calculations, access data, and take actions beyond text generation.

## What you'll learn
- Creating custom tools with the `@tool` decorator
- Using built-in tools from `strands-agents-tools`
- Error handling and validation in tools
- Combining multiple tools for complex workflows

## Prerequisites
- Completed notebook 01 (Basic Agent Creation)
- Understanding of Python functions and decorators

## Setup and Configuration

In [1]:
!pip install -r requirements.txt

In [2]:
import os
import json
import datetime
import random
import requests
from strands import Agent, tool
from strands_tools import http_request
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.WARNING)

# Consistent model configuration
DEFAULT_MODEL = "us.anthropic.claude-sonnet-4-20250514-v1:0"
AWS_REGION = "us-west-2"

print("Environment configured for tools workshop")

Environment configured for tools workshop


## Understanding Tools

Tools are Python functions that agents can call during their reasoning process. They extend agent capabilities beyond text generation.

### Your First Custom Tool

In [ ]:
@tool
def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Test the tool directly
print("Current time:", get_current_time())

Now let's create an agent that can use this tool:

In [ ]:
# Create an agent with the time tool
time_agent = Agent(
    model=DEFAULT_MODEL,
    tools=[get_current_time],
    system_prompt="You are a helpful assistant that can check the current time when needed."
)

# Test the agent
response = time_agent("What time is it right now?")
print(response)

## Creating Practical Tools

Let's build tools that solve real problems:

In [ ]:
@tool
def calculate(expression: str) -> str:
    """Safely evaluate mathematical expressions.
    
    Args:
        expression: Mathematical expression to evaluate (e.g., "2 + 3 * 4")
    
    Returns:
        Result of the calculation or error message
    """
    try:
        # Only allow safe mathematical operations
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def generate_random_number(min_val: int = 1, max_val: int = 100) -> int:
    """Generate a random number within a specified range.
    
    Args:
        min_val: Minimum value (default: 1)
        max_val: Maximum value (default: 100)
    
    Returns:
        Random integer between min_val and max_val (inclusive)
    """
    return random.randint(min_val, max_val)

# Test the tools
print("Calculation:", calculate("15 * 3 + 7"))
print("Random number:", generate_random_number(1, 10))

## Agent with Multiple Tools

Let's create an agent that can use multiple tools together:

In [ ]:
# Create a multi-tool agent
utility_agent = Agent(
    model=DEFAULT_MODEL,
    tools=[get_current_time, calculate, generate_random_number],
    system_prompt="""
    You are a utility assistant with access to time, calculation, and random number tools.
    Use the appropriate tools to help users with their requests.
    Always explain what you're doing when using tools.
    """
)

# Test with different requests
test_queries = [
    "What's the current time and calculate 25 * 4?",
    "Generate a random number between 50 and 100",
    "Calculate the area of a circle with radius 5 (use π ≈ 3.14159)"
]

for query in test_queries:
    print(f"\nUser: {query}")
    response = utility_agent(query)
    print(f"Assistant: {response}")
    print("-" * 50)

## Working with External APIs

Tools can interact with external services. Let's create a weather tool:

In [ ]:
@tool
def get_weather_info(city: str) -> str:
    """Get current weather information for a city.
    
    Args:
        city: Name of the city
    
    Returns:
        Weather information or error message
    """
    try:
        # Using a free weather API (OpenWeatherMap)
        # Note: In production, you'd use a real API key
        api_key = os.getenv('OPENWEATHER_API_KEY')
        
        if not api_key:
            # Return mock data for demonstration
            return f"Mock weather data for {city}: 22°C, partly cloudy, humidity 65%"
        
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
        response = requests.get(url, timeout=5)
        
        if response.status_code == 200:
            data = response.json()
            temp = data['main']['temp']
            description = data['weather'][0]['description']
            humidity = data['main']['humidity']
            return f"Weather in {city}: {temp}°C, {description}, humidity {humidity}%"
        else:
            return f"Could not fetch weather data for {city}"
            
    except Exception as e:
        return f"Error fetching weather: {str(e)}"

# Test the weather tool
print(get_weather_info("London"))

## File Operations Tool

Let's create a tool for basic file operations:

In [ ]:
@tool
def save_text_to_file(filename: str, content: str) -> str:
    """Save text content to a file.
    
    Args:
        filename: Name of the file to create
        content: Text content to save
    
    Returns:
        Success or error message
    """
    try:
        # Basic security: only allow certain file extensions
        allowed_extensions = ['.txt', '.md', '.json', '.csv']
        if not any(filename.endswith(ext) for ext in allowed_extensions):
            return "Error: Only .txt, .md, .json, and .csv files are allowed"
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        
        return f"Successfully saved content to {filename}"
    except Exception as e:
        return f"Error saving file: {str(e)}"

@tool
def read_text_file(filename: str) -> str:
    """Read content from a text file.
    
    Args:
        filename: Name of the file to read
    
    Returns:
        File content or error message
    """
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()
        return f"Content of {filename}:\n{content}"
    except FileNotFoundError:
        return f"Error: File {filename} not found"
    except Exception as e:
        return f"Error reading file: {str(e)}"

# Test file operations
test_content = "This is a test file created by the Strands agent."
print(save_text_to_file("test_output.txt", test_content))
print(read_text_file("test_output.txt"))

## Advanced Tool Patterns

### Tool with Complex Data Structures

In [ ]:
@tool
def analyze_data(data_json: str) -> str:
    """Analyze a dataset provided as JSON.
    
    Args:
        data_json: JSON string containing numerical data
    
    Returns:
        Statistical analysis of the data
    """
    try:
        data = json.loads(data_json)
        
        if not isinstance(data, list) or not all(isinstance(x, (int, float)) for x in data):
            return "Error: Data must be a JSON array of numbers"
        
        if not data:
            return "Error: Data array is empty"
        
        # Calculate statistics
        count = len(data)
        total = sum(data)
        mean = total / count
        minimum = min(data)
        maximum = max(data)
        
        # Calculate median
        sorted_data = sorted(data)
        if count % 2 == 0:
            median = (sorted_data[count//2 - 1] + sorted_data[count//2]) / 2
        else:
            median = sorted_data[count//2]
        
        return f"""
Data Analysis Results:
- Count: {count}
- Sum: {total}
- Mean: {mean:.2f}
- Median: {median}
- Min: {minimum}
- Max: {maximum}
- Range: {maximum - minimum}
""".strip()
        
    except json.JSONDecodeError:
        return "Error: Invalid JSON format"
    except Exception as e:
        return f"Error analyzing data: {str(e)}"

# Test the data analysis tool
sample_data = '[10, 15, 20, 25, 30, 35, 40]'
print(analyze_data(sample_data))

## Creating a Comprehensive Assistant

Let's combine all our tools into a powerful assistant:

In [ ]:
# Create a comprehensive assistant with all tools
comprehensive_assistant = Agent(
    model=DEFAULT_MODEL,
    tools=[
        get_current_time,
        calculate,
        generate_random_number,
        get_weather_info,
        save_text_to_file,
        read_text_file,
        analyze_data
    ],
    system_prompt="""
    You are a comprehensive assistant with access to multiple tools:
    - Time and date information
    - Mathematical calculations
    - Random number generation
    - Weather information
    - File operations (read/write)
    - Data analysis
    
    Use the appropriate tools to help users accomplish their tasks.
    Always explain what you're doing and why you're using specific tools.
    """
)

print("Comprehensive assistant created with 7 tools")

Let's test the comprehensive assistant with complex requests:

In [ ]:
# Test complex workflow
complex_request = """
I need you to:
1. Generate 5 random numbers between 1 and 100
2. Calculate their average
3. Save the results to a file called 'random_analysis.txt'
4. Include the current timestamp in the file
"""

response = comprehensive_assistant(complex_request)
print(response)

## Error Handling and Validation

Robust tools include proper error handling:

In [ ]:
@tool
def robust_calculator(operation: str, a: float, b: float) -> str:
    """Perform mathematical operations with comprehensive error handling.
    
    Args:
        operation: Operation to perform (add, subtract, multiply, divide)
        a: First number
        b: Second number
    
    Returns:
        Result of the operation or detailed error message
    """
    # Input validation
    valid_operations = ['add', 'subtract', 'multiply', 'divide']
    
    if operation.lower() not in valid_operations:
        return f"Error: Invalid operation '{operation}'. Valid operations: {', '.join(valid_operations)}"
    
    try:
        operation = operation.lower()
        
        if operation == 'add':
            result = a + b
        elif operation == 'subtract':
            result = a - b
        elif operation == 'multiply':
            result = a * b
        elif operation == 'divide':
            if b == 0:
                return "Error: Division by zero is not allowed"
            result = a / b
        
        return f"Result: {a} {operation} {b} = {result}"
        
    except TypeError:
        return "Error: Both arguments must be numbers"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

# Test error handling
print(robust_calculator("divide", 10, 2))
print(robust_calculator("divide", 10, 0))
print(robust_calculator("invalid", 5, 3))

## Using Built-in Tools

Strands provides built-in tools through the `strands-agents-tools` package:

In [ ]:
# Agent with built-in HTTP request tool
web_agent = Agent(
    model=DEFAULT_MODEL,
    tools=[http_request],
    system_prompt="""
    You can make HTTP requests to web APIs.
    Always explain what you're doing when making requests.
    Be careful with external APIs and handle errors gracefully.
    """
)

# Test with a public API
api_request = "Can you get information from https://httpbin.org/json?"
response = web_agent(api_request)
print(response)

## Best Practices for Tool Development

Based on production experience, here are key guidelines:

In [ ]:
# Example of a well-designed production tool
@tool
def production_ready_tool(input_data: str, options: str = "") -> str:
    """Example of a production-ready tool with all best practices.
    
    Args:
        input_data: Primary input data
        options: Optional configuration (JSON string)
    
    Returns:
        Processed result or error message
    """
    # 1. Input validation
    if not input_data or not isinstance(input_data, str):
        return "Error: input_data must be a non-empty string"
    
    # 2. Parse options safely
    config = {}
    if options:
        try:
            config = json.loads(options)
        except json.JSONDecodeError:
            return "Error: options must be valid JSON"
    
    # 3. Set defaults
    max_length = config.get('max_length', 1000)
    case_sensitive = config.get('case_sensitive', True)
    
    try:
        # 4. Main processing logic
        processed_data = input_data[:max_length]
        if not case_sensitive:
            processed_data = processed_data.lower()
        
        # 5. Return structured result
        result = {
            'original_length': len(input_data),
            'processed_length': len(processed_data),
            'truncated': len(input_data) > max_length,
            'result': processed_data
        }
        
        return json.dumps(result, indent=2)
        
    except Exception as e:
        # 6. Comprehensive error handling
        return f"Processing error: {str(e)}"

# Test the production-ready tool
test_input = "This is a test string for the production tool"
test_options = '{"max_length": 20, "case_sensitive": false}'
print(production_ready_tool(test_input, test_options))

## Key Takeaways

You've learned how to extend agent capabilities with tools:

**Core Concepts:**
- Tools are Python functions decorated with `@tool`
- Agents can use multiple tools in combination
- Tools enable agents to interact with external systems

**Best Practices:**
- Always include comprehensive docstrings
- Implement robust error handling
- Validate inputs thoroughly
- Return structured, informative results
- Consider security implications

**Production Considerations:**
- Rate limiting for external API calls
- Logging and monitoring
- Input sanitization
- Timeout handling

## Practice Exercise

Create your own tool for a specific use case:

In [ ]:
# Your turn: Create a custom tool
# Ideas: URL shortener, password generator, unit converter, etc.

@tool
def my_custom_tool(param1: str, param2: int = 10) -> str:
    """Your custom tool description.
    
    Args:
        param1: Description of first parameter
        param2: Description of second parameter
    
    Returns:
        Description of return value
    """
    # Implement your tool logic here
    return f"Processed {param1} with value {param2}"

# Test your tool
# print(my_custom_tool("test", 5))

# Create an agent with your tool
# my_agent = Agent(model=DEFAULT_MODEL, tools=[my_custom_tool])
# response = my_agent("Use my custom tool with some test data")
# print(response)